In [39]:
# Import all the required libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline

In [40]:
# Load the data from the CSV file into a dataframe

loc = '/Users/prashanth/Downloads/DS Project/'

laliga_1 = pd.read_csv(loc + '2019-20.csv')
laliga_2 = pd.read_csv(loc + '2020-21.csv')
laliga_3 = pd.read_csv(loc + '2021-22.csv')
laliga_4 = pd.read_csv(loc + '2022-23.csv')
laliga_5 = pd.read_csv(loc + '2023-24.csv')


In [41]:

from datetime import datetime as dt

def date_parsing(date):
    if date == '':
        return None
    for fmt in ('%d/%m/%Y', '%d/%m/%y'):
        try:
            return dt.strptime(date, fmt).date()
        except ValueError:
            continue
    raise ValueError(f"Date format for {date} not recognized")

# Apply the date parsing to all dataframes
laliga_1.Date = laliga_1.Date.apply(date_parsing)    
laliga_2.Date = laliga_2.Date.apply(date_parsing)    
laliga_3.Date = laliga_3.Date.apply(date_parsing)          
laliga_4.Date = laliga_4.Date.apply(date_parsing)    
laliga_5.Date = laliga_5.Date.apply(date_parsing)

# Extract relevant statistics
required_columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
performance_stats_1 = laliga_1[required_columns]
performance_stats_2 = laliga_2[required_columns]
performance_stats_3 = laliga_3[required_columns]
performance_stats_4 = laliga_4[required_columns]
performance_stats_5 = laliga_5[required_columns]









In [27]:
 # Get the aggregated goals scored, organized by teams and matchweek

def get_goals_scored(performance_stats):
    # Construct a dictionary where the team names serve as the keys.
    teams = {}
    for i in performance_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # The value for each key is a list that includes the match location.
    for i in range(len(performance_stats)):
        HTGS = performance_stats.iloc[i]['FTHG']
        ATGS = performance_stats.iloc[i]['FTAG']
        teams[performance_stats.iloc[i].HomeTeam].append(HTGS)
        teams[performance_stats.iloc[i].AwayTeam].append(ATGS)
    
   # Generate a dataframe for goals scored with teams as rows and matchweeks as columns.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get the cumulative totals up to that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



# Get the aggregated goals conceded, organized by teams and matchweek
def get_goals_conceded(performance_stats):
    # Construct a dictionary where team names are used as keys
    
    teams = {}
    for i in performance_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # The value associated with each key is a list that includes the match location.
    for i in range(len(performance_stats)):
        ATGC = performance_stats.iloc[i]['FTHG']
        HTGC = performance_stats.iloc[i]['FTAG']
        teams[performance_stats.iloc[i].HomeTeam].append(HTGC)
        teams[performance_stats.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored with teams as rows and matchweeks as columns.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    
    # Aggregate to obtain cumulative totals up to that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def get_gss(performance_stats):
    GC = get_goals_conceded(performance_stats)
    GS = get_goals_scored(performance_stats)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = performance_stats.iloc[i].HomeTeam
        at = performance_stats.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    performance_stats['HTGS'] = HTGS
    performance_stats['ATGS'] = ATGS
    performance_stats['HTGC'] = HTGC
    performance_stats['ATGC'] = ATGC
    
    return performance_stats


# Apply to each dataset
performance_stats_1 = get_gss(performance_stats_1)
performance_stats_2 = get_gss(performance_stats_2)
performance_stats_3 = get_gss(performance_stats_3)
performance_stats_4 = get_gss(performance_stats_4)
performance_stats_5 = get_gss(performance_stats_5)




/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/885029972.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['HTGS'] = HTGS
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/885029972.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['ATGS'] = ATGS
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/885029972.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [28]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2, 39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column=0, loc=0, value=[0]*len(matchres_points))
    return matchres_points


def get_matchres(performance_stats):
    # Create a dictionary with team names as keys
    teams = {}
    for i in performance_stats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(performance_stats)):
        if performance_stats.iloc[i].FTR == 'H':
            teams[performance_stats.iloc[i].HomeTeam].append('W')
            teams[performance_stats.iloc[i].AwayTeam].append('L')
        elif performance_stats.iloc[i].FTR == 'A':
            teams[performance_stats.iloc[i].AwayTeam].append('W')
            teams[performance_stats.iloc[i].HomeTeam].append('L')
        else:
            teams[performance_stats.iloc[i].AwayTeam].append('D')
            teams[performance_stats.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index=[i for i in range(1, 39)]).T

def get_agg_points(performance_stats):
    matchres = get_matchres(performance_stats)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = performance_stats.iloc[i].HomeTeam
        at = performance_stats.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1) % 10) == 0:
            j = j + 1
            
    performance_stats['HTP'] = HTP
    performance_stats['ATP'] = ATP
    return performance_stats
    
# Apply to each dataset
performance_stats_1 = get_agg_points(performance_stats_1)
performance_stats_2 = get_agg_points(performance_stats_2)
performance_stats_3 = get_agg_points(performance_stats_3)
performance_stats_4 = get_agg_points(performance_stats_4)
performance_stats_5 = get_agg_points(performance_stats_5)


/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/3049378596.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['HTP'] = HTP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/3049378596.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['ATP'] = ATP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/3049378596.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [30]:
def get_form(performance_stats,num):
    form = get_matchres(performance_stats)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(performance_stats,num):
    form = get_form(performance_stats,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = performance_stats.iloc[i].HomeTeam
        at = performance_stats.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    performance_stats['HM' + str(num)] = h                 
    performance_stats['AM' + str(num)] = a

    
    return performance_stats


def add_form_df(performance_stats):
    performance_stats = add_form(performance_stats,1)
    performance_stats = add_form(performance_stats,2)
    performance_stats = add_form(performance_stats,3)
    performance_stats = add_form(performance_stats,4)
    performance_stats = add_form(performance_stats,5)
    return performance_stats    
    
# Make changes to df
performance_stats_1 = add_form_df(performance_stats_1)
performance_stats_2 = add_form_df(performance_stats_2)
performance_stats_3 = add_form_df(performance_stats_3)
performance_stats_4 = add_form_df(performance_stats_4)
performance_stats_5 = add_form_df(performance_stats_5)



/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/685001438.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['HM' + str(num)] = h
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/685001438.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['AM' + str(num)] = a
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/685001438.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [31]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

performance_stats_1 = performance_stats_1[cols]
performance_stats_2 = performance_stats_2[cols]
performance_stats_3 = performance_stats_3[cols]
performance_stats_4 = performance_stats_4[cols]
performance_stats_5 = performance_stats_5[cols]


In [32]:
Standings = pd.read_csv(loc + "Laliga_Standings.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)

def get_last(performance_stats, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = performance_stats.iloc[i].HomeTeam
        at = performance_stats.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    performance_stats['HomeTeamLP'] = HomeTeamLP
    performance_stats['AwayTeamLP'] = AwayTeamLP
    return performance_stats

performance_stats_1 = get_last(performance_stats_1, Standings, 0)
performance_stats_2 = get_last(performance_stats_2, Standings, 1)
performance_stats_3 = get_last(performance_stats_3, Standings, 2)
performance_stats_4 = get_last(performance_stats_4, Standings, 3)
performance_stats_5 = get_last(performance_stats_5, Standings, 4)


/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/2446221286.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['HomeTeamLP'] = HomeTeamLP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_1381/2446221286.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_stats['AwayTeamLP'] = AwayTeamLP


In [33]:
def get_mw(performance_stats):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    performance_stats['MW'] = MatchWeek
    return performance_stats

performance_stats_1 = get_mw(performance_stats_1)
performance_stats_2 = get_mw(performance_stats_2)
performance_stats_3 = get_mw(performance_stats_3)
performance_stats_4 = get_mw(performance_stats_4)
performance_stats_5 = get_mw(performance_stats_5)


In [34]:
performance_stats = pd.concat([performance_stats_1,
                          performance_stats_2,
                          performance_stats_3,
                          performance_stats_4,
                          performance_stats_5,], ignore_index=True)


# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

performance_stats['HTFormPtsStr'] = performance_stats['HM1'] + performance_stats['HM2'] + performance_stats['HM3'] + performance_stats['HM4'] + performance_stats['HM5']
performance_stats['ATFormPtsStr'] = performance_stats['AM1'] + performance_stats['AM2'] + performance_stats['AM3'] + performance_stats['AM4'] + performance_stats['AM5']

performance_stats['HTFormPts'] = performance_stats['HTFormPtsStr'].apply(get_form_points)
performance_stats['ATFormPts'] = performance_stats['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
performance_stats['HTWinStreak3'] = performance_stats['HTFormPtsStr'].apply(get_3game_ws)
performance_stats['HTWinStreak5'] = performance_stats['HTFormPtsStr'].apply(get_5game_ws)
performance_stats['HTLossStreak3'] = performance_stats['HTFormPtsStr'].apply(get_3game_ls)
performance_stats['HTLossStreak5'] = performance_stats['HTFormPtsStr'].apply(get_5game_ls)

performance_stats['ATWinStreak3'] = performance_stats['ATFormPtsStr'].apply(get_3game_ws)
performance_stats['ATWinStreak5'] = performance_stats['ATFormPtsStr'].apply(get_5game_ws)
performance_stats['ATLossStreak3'] = performance_stats['ATFormPtsStr'].apply(get_3game_ls)
performance_stats['ATLossStreak5'] = performance_stats['ATFormPtsStr'].apply(get_5game_ls)

performance_stats.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [35]:
# Calculate Goal Difference
performance_stats['HTGD'] = performance_stats['HTGS'] - performance_stats['HTGC']
performance_stats['ATGD'] = performance_stats['ATGS'] - performance_stats['ATGC']

# Calculate Differences
performance_stats['DiffPts'] = performance_stats['HTP'] - performance_stats['ATP']
performance_stats['DiffFormPts'] = performance_stats['HTFormPts'] - performance_stats['ATFormPts']

# Difference in Last Year's Positions
performance_stats['DiffLP'] = performance_stats['HomeTeamLP'] - performance_stats['AwayTeamLP']

# Display the first few rows to verify
performance_stats.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,2019-08-16,Ath Bilbao,Barcelona,1,0,H,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.0
1,2019-08-17,Celta,Real Madrid,1,3,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16.0
2,2019-08-17,Valencia,Sociedad,1,1,D,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,2019-08-17,Mallorca,Eibar,2,1,H,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0
4,2019-08-17,Leganes,Osasuna,0,1,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.0


In [36]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
performance_stats.MW = performance_stats.MW.astype(float)

for col in cols:
    performance_stats[col] = performance_stats[col] / performance_stats.MW

In [37]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
performance_stats['FTR'] = performance_stats.FTR.apply(only_hw)

# Testing set (2015-16 season)
performance_stats_test = performance_stats[1520:]

In [38]:
performance_stats.to_csv(loc + "final_dataset.csv")
performance_stats_test.to_csv(loc+"test.csv")